## Agente para responder preguntas de tu hv

### Y, uso de la herramienta.

### Pushover

Pushover es una herramienta práctica para enviar notificaciones push a tu teléfono.

¡Es facilísima de configurar e instalar!

Simplemente visita https://pushover.net/, crea una cuenta gratuita y genera tus claves API.

Como señaló el estudiante Ron (¡gracias, Ron!), hay dos tokens que se pueden crear en Pushover:
1. El token de usuario, que se obtiene en la página principal de Pushover.
2. El token de aplicación, que se obtiene al ir a https://pushover.net/apps/build y crear una aplicación.

(Esto te permite organizar tus notificaciones push en diferentes aplicaciones en el futuro).

Agrega a tu archivo `.env`:
```
PUSHOVER_USER=pon_tu_token_de_usuario_aquí
PUSHOVER_TOKEN=pon_tu_token_de_aplicación_aquí
```

E instala la aplicación Pushover en tu teléfono.

In [2]:
# imports

from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, PageBreak, ListFlowable, ListItem
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.units import inch
from reportlab.pdfgen import canvas
from io import BytesIO

In [3]:
# El inicio usual

load_dotenv(override=True)
openai = OpenAI()

In [4]:
# Para pushover
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

# Validar que las variables de entorno estén configuradas (opcional, pero recomendado)
if not pushover_user or not pushover_token:
    print("⚠️ Advertencia: PUSHOVER_USER o PUSHOVER_TOKEN no están configurados en .env")
    print("   Las notificaciones push no funcionarán hasta que se configuren.")

In [5]:
def push(message):
    """
    Envía una notificación push usando Pushover.
    
    Args:
        message: El mensaje a enviar
        
    Returns:
        bool: True si se envió correctamente, False en caso contrario
    """
    if not pushover_user or not pushover_token:
        print(f"⚠️ Push no configurado: {message}")
        return False
    
    try:
        print(f"Push: {message}")
        payload = {"user": pushover_user, "token": pushover_token, "message": message}
        response = requests.post(pushover_url, data=payload, timeout=10)
        response.raise_for_status()
        return True
    except requests.exceptions.RequestException as e:
        print(f"❌ Error enviando push: {e}")
        return False

In [6]:
push("HOLA!!")

Push: HOLA!!


True

In [8]:
# Cargar perfil de LinkedIn
try:
    reader = PdfReader("me/linkedinandres.pdf")
    linkedin = ""
    for page in reader.pages:
        text = page.extract_text()
        if text:
            linkedin += text
except FileNotFoundError:
    print("⚠️ Archivo me/linkedinandres.pdf no encontrado")
    linkedin = ""
except Exception as e:
    print(f"❌ Error leyendo PDF: {e}")
    linkedin = ""

# Cargar resumen
try:
    with open("me/summary.txt", "r", encoding="utf-8") as f:
        summary = f.read()
except FileNotFoundError:
    print("⚠️ Archivo me/summary.txt no encontrado")
    summary = ""
except Exception as e:
    print(f"❌ Error leyendo summary: {e}")
    summary = ""

name = "Andres Bedoya"

In [9]:
def record_user_details(email, name="Nombre no proporcionado", notes="not provided"):
    """
    Registra los detalles de un usuario interesado.
    
    Args:
        email: Dirección de correo electrónico (requerido)
        name: Nombre del usuario
        notes: Notas adicionales sobre la conversación
        
    Returns:
        dict: Estado de la operación
    """
    if not email or not email.strip():
        return {"recorded": "error", "message": "Email es requerido"}
    
    push(f"Registrando interés de {name} con email {email} y notas {notes}")
    return {"recorded": "ok"}

In [10]:
def record_unknown_question(question):
    """
    Registra una pregunta que no pudo ser respondida.
    
    Args:
        question: La pregunta que no se pudo responder
        
    Returns:
        dict: Estado de la operación
    """
    if not question or not question.strip():
        return {"recorded": "error", "message": "La pregunta no puede estar vacía"}
    
    push(f"Registrando pregunta no respondida: {question}")
    return {"recorded": "ok"}

In [12]:
def generate_cv_pdf(job_description):
    """
    Genera un PDF optimizado para ATS basado en el perfil de Andrés Bedoya y el Job Description.
    
    Args:
        job_description: Descripción del trabajo para adaptar el CV
        
    Returns:
        dict: Ruta del PDF generado y estado de la operación
    """
    if not job_description or not job_description.strip():
        return {"pdf_path": None, "status": "error", "message": "Job description es requerido"}
    
    try:
        # Llamada al modelo para generar texto adaptado
        prompt = f"""
        Eres un experto en redacción de CVs y optimización ATS.
        Usa el siguiente perfil profesional y el Job Description para generar una hoja de vida adaptada:
        
        PERFIL:
        {summary}
        
        LINKEDIN:
        {linkedin}
        
        JOB DESCRIPTION:
        {job_description}
        
        Requisitos:
        - Usa palabras clave del JD.
        - Reescribe el resumen profesional y la experiencia de manera relevante.
        - No inventes logros.
        - Usa formato limpio, sin tablas ni columnas.
        - Incluye secciones: Perfil Profesional, Experiencia, Habilidades, Educación.
        """

        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": prompt}],
        )

        cv_text = response.choices[0].message.content

        # Generar PDF en memoria
        buffer = BytesIO()
        c = canvas.Canvas(buffer, pagesize=letter)
        text_obj = c.beginText(40, 750)
        text_obj.setFont("Helvetica", 11)

        for line in cv_text.split("\n"):
            text_obj.textLine(line.strip())
        c.drawText(text_obj)
        c.save()

        # Guardar PDF
        output_path = "me/cv_ats_ready.pdf"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with open(output_path, "wb") as f:
            f.write(buffer.getvalue())

        push(f"✅ CV optimizado ATS generado: {output_path}")
        return {"pdf_path": output_path, "status": "ok"}
    except Exception as e:
        error_msg = f"Error generando CV: {str(e)}"
        print(f"❌ {error_msg}")
        push(f"❌ {error_msg}")
        return {"pdf_path": None, "status": "error", "message": error_msg}


In [13]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Utilice esta herramienta para registrar que un usuario está interesado en estar en contacto y proporcionó una dirección de correo electrónico.",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "La dirección de correo electrónico de este usuario"
            },
            "name": {
                "type": "string",
                "description": "El nombre del usuario, si lo proporcionó"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Cualquier información adicional sobre la conversación que merezca ser registrada para dar contexto"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

In [14]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Siempre use esta herramienta para registrar cualquier pregunta que no se pueda responder, ya que no sabía la respuesta",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "La pregunta que no se pudo responder"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [19]:
generate_cv_pdf_json = {
    "name": "generate_cv_pdf_premium",
    "description": "Genera un nuevo PDF de CV optimizado para un Job Description dado.",
    "parameters": {
        "type": "object",
        "properties": {
            "job_description": {
                "type": "string",
                "description": "El texto completo del Job Description para adaptar la hoja de vida."
            }
        },
        "required": ["job_description"],
        "additionalProperties": False
    }
}



In [20]:
def generate_cv_pdf_premium(job_description):
    """
    Genera un CV en PDF con diseño profesional, optimizado para ATS y legible para reclutadores.
    
    Args:
        job_description: Descripción del trabajo para adaptar el CV
        
    Returns:
        dict: Ruta del PDF generado y estado de la operación
    """
    if not job_description or not job_description.strip():
        return {"pdf_path": None, "status": "error", "message": "Job description es requerido"}
    
    try:
        # 1️⃣ Generar CV adaptado con LLM
        prompt = f"""
Eres un experto en redacción de CVs y optimización ATS.
Genera un CV adaptado al siguiente Job Description:

PERFIL:
{summary}

LINKEDIN:
{linkedin}

JOB DESCRIPTION:
{job_description}

Requisitos:
- Organiza en secciones: PERFIL, EXPERIENCIA, HABILIDADES, EDUCACIÓN.
- Usa bullets para logros y habilidades.
- Optimizado para ATS (palabras clave del JD).
- Formato profesional, limpio, con títulos claros y jerarquía visual.
- Separa logros y responsabilidades de cada experiencia.
- Cada sección debe empezar en una línea nueva y con título destacado.
"""

        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": prompt}],
        )

        cv_text = response.choices[0].message.content

        # 2️⃣ Crear PDF con Platypus
        buffer = BytesIO()
        doc = SimpleDocTemplate(buffer, pagesize=letter,
                                rightMargin=50, leftMargin=50,
                                topMargin=50, bottomMargin=50)

        styles = getSampleStyleSheet()
        story = []

        # Encabezado: nombre y LinkedIn
        header_style = ParagraphStyle(
            'Header',
            parent=styles['Heading1'],
            fontSize=20,
            leading=24,
            spaceAfter=12
        )
        story.append(Paragraph(name, header_style))

        link_style = ParagraphStyle(
            'Link',
            parent=styles['Normal'],
            textColor=colors.blue,
            fontSize=12,
            spaceAfter=15
        )
        story.append(Paragraph(linkedin, link_style))

        # Estilos para secciones y bullets
        section_style = ParagraphStyle(
            'Section',
            parent=styles['Heading2'],
            fontSize=14,
            textColor=colors.darkblue,
            spaceBefore=12,
            spaceAfter=6,
            leading=16
        )
        bullet_style = ParagraphStyle(
            'Bullet',
            parent=styles['Normal'],
            leftIndent=20,
            bulletIndent=10,
            spaceAfter=3,
            fontSize=11,
            leading=14
        )

        # 3️⃣ Procesar el texto del CV y organizar en secciones
        current_section = None
        bullets = []
        for line in cv_text.split("\n"):
            line = line.strip()
            if not line:
                continue
            if line.endswith(":"):  # sección
                if bullets:  # agregar bullets anteriores
                    story.append(ListFlowable(bullets, bulletType='bullet'))
                    bullets = []
                current_section = line[:-1].upper()
                story.append(Paragraph(current_section, section_style))
            elif line.startswith("-"):  # bullet
                bullets.append(ListItem(Paragraph(line[1:].strip(), bullet_style)))
            else:  # texto normal dentro de sección
                if bullets:
                    story.append(ListFlowable(bullets, bulletType='bullet'))
                    bullets = []
                story.append(Paragraph(line, styles['Normal']))

        # agregar bullets restantes
        if bullets:
            story.append(ListFlowable(bullets, bulletType='bullet'))

        # 4️⃣ Construir PDF
        doc.build(story)

        # Guardar PDF
        output_path = "me/cv_ats_premium.pdf"
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with open(output_path, "wb") as f:
            f.write(buffer.getvalue())

        push(f"✅ CV premium optimizado ATS generado: {output_path}")
        return {"pdf_path": output_path, "status": "ok"}
    except Exception as e:
        error_msg = f"Error generando CV premium: {str(e)}"
        print(f"❌ {error_msg}")
        push(f"❌ {error_msg}")
        return {"pdf_path": None, "status": "error", "message": error_msg}

In [21]:
tools = [
    {"type": "function", "function": record_user_details_json},
    {"type": "function", "function": record_unknown_question_json},
    {"type": "function", "function": generate_cv_pdf_json}
]

In [22]:
tools

[{'type': 'function',
  'function': {'name': 'record_user_details',
   'description': 'Utilice esta herramienta para registrar que un usuario está interesado en estar en contacto y proporcionó una dirección de correo electrónico.',
   'parameters': {'type': 'object',
    'properties': {'email': {'type': 'string',
      'description': 'La dirección de correo electrónico de este usuario'},
     'name': {'type': 'string',
      'description': 'El nombre del usuario, si lo proporcionó'},
     'notes': {'type': 'string',
      'description': 'Cualquier información adicional sobre la conversación que merezca ser registrada para dar contexto'}},
    'required': ['email'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question',
   'description': 'Siempre use esta herramienta para registrar cualquier pregunta que no se pueda responder, ya que no sabía la respuesta',
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 

In [23]:
# Esta función puede tomar una lista de llamadas a herramientas y ejecutarlas. ¡Este es el IF statement!!

def handle_tool_calls(tool_calls):
    """
    Ejecuta las llamadas a herramientas solicitadas por el LLM.
    
    Args:
        tool_calls: Lista de llamadas a herramientas
        
    Returns:
        list: Lista de resultados de las herramientas ejecutadas
    """
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        try:
            arguments = json.loads(tool_call.function.arguments)
        except json.JSONDecodeError as e:
            print(f"❌ Error parseando argumentos de {tool_name}: {e}", flush=True)
            results.append({
                "role": "tool",
                "content": json.dumps({"error": f"Invalid arguments: {str(e)}"}),
                "tool_call_id": tool_call.id
            })
            continue
        
        print(f"Herramienta llamada: {tool_name}", flush=True)

        # ¡EL GRAN IF !!!
        try:
            if tool_name == "record_user_details":
                result = record_user_details(**arguments)
            elif tool_name == "record_unknown_question":
                result = record_unknown_question(**arguments)
            elif tool_name == "generate_cv_pdf":
                result = generate_cv_pdf_premium(**arguments)
            else:
                result = {"error": f"Herramienta desconocida: {tool_name}"}
                print(f"⚠️ Herramienta desconocida: {tool_name}", flush=True)
        except Exception as e:
            error_msg = f"Error ejecutando {tool_name}: {str(e)}"
            print(f"❌ {error_msg}", flush=True)
            result = {"error": error_msg}

        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })
    return results

In [30]:
globals()["generate_cv_pdf_premium"]("se necesita un arquitecto de soluciones con experiencia en cloud y java")

Push: ✅ CV premium optimizado ATS generado: me/cv_ats_premium.pdf


{'pdf_path': 'me/cv_ats_premium.pdf', 'status': 'ok'}

In [25]:
# Esta es una forma más elegante de evitar el IF statement.
# Nota: Usar globals() puede ser menos seguro, pero es más flexible
def handle_tool_calls(tool_calls):
    """
    Ejecuta las llamadas a herramientas usando globals() para evitar IF statements.
    Nota: Esta versión es más flexible pero menos explícita que la versión con IF.
    
    Args:
        tool_calls: Lista de llamadas a herramientas
        
    Returns:
        list: Lista de resultados de las herramientas ejecutadas
    """
    # Mapeo explícito de herramientas permitidas (más seguro que globals())
    allowed_tools = {
        "record_user_details": record_user_details,
        "record_unknown_question": record_unknown_question,
        "generate_cv_pdf": generate_cv_pdf
    }
    
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        try:
            arguments = json.loads(tool_call.function.arguments)
        except json.JSONDecodeError as e:
            print(f"❌ Error parseando argumentos de {tool_name}: {e}", flush=True)
            results.append({
                "role": "tool",
                "content": json.dumps({"error": f"Invalid arguments: {str(e)}"}),
                "tool_call_id": tool_call.id
            })
            continue
        
        print(f"Herramienta llamada: {tool_name}", flush=True)
        
        tool = allowed_tools.get(tool_name)
        if tool:
            try:
                result = tool(**arguments)
            except Exception as e:
                error_msg = f"Error ejecutando {tool_name}: {str(e)}"
                print(f"❌ {error_msg}", flush=True)
                result = {"error": error_msg}
        else:
            result = {"error": f"Herramienta desconocida: {tool_name}"}
            print(f"⚠️ Herramienta desconocida: {tool_name}", flush=True)
        
        results.append({
            "role": "tool",
            "content": json.dumps(result),
            "tool_call_id": tool_call.id
        })
    return results

In [26]:
system_prompt = f"""Estás actuando como {name}. Estás respondiendo preguntas en el sitio web de {name}, en particular preguntas relacionadas con la carrera, los antecedentes, las habilidades y la experiencia de {name}.
Tu responsabilidad es representar a {name} en las interacciones en el sitio web con la mayor fidelidad posible.
Se te proporciona un resumen de los antecedentes y el perfil de LinkedIn de {name} que puedes usar para responder preguntas.
Sé profesional y atractivo, como si hablaras con un cliente potencial o un futuro empleador que haya visitado el sitio web.
Si no sabes la respuesta a alguna pregunta, usa la herramienta record_unknown_question para registrar la pregunta que no pudiste responder, incluso si se trata de algo trivial o no relacionado con tu carrera.
Si el usuario participa en una conversación, intenta que se ponga en contacto por correo electrónico; pídele su correo electrónico y regístralo con la herramienta record_user_details."""

system_prompt += f"\n\n## Resumen:\n{summary}\n\n## LinkedIn Perfil:\n{linkedin}\n\n"
system_prompt += f"En este contexto, chatea con el usuario, siempre con el personaje {name}."

In [27]:
system_prompt = f"""
Actúas como un asistente profesional experto en recursos humanos y optimización de CVs.
Tu tarea principal es ayudar a {name} a generar versiones personalizadas de su hoja de vida según cada Job Description (JD).

Cuando el usuario proporcione un JD, analiza sus requisitos y genera una nueva hoja de vida adaptada a dicho rol.
Si el usuario solicita el PDF, usa la herramienta `generate_cv_pdf`.

Además, si el usuario hace preguntas sobre la carrera de {name}, responde profesionalmente usando su perfil.

### Perfil de referencia:
Resumen:
{summary}

LinkedIn:
{linkedin}

Si no sabes la respuesta a una pregunta, usa la herramienta record_unknown_question.
Si el usuario desea contacto, usa record_user_details.
"""


In [28]:
def chat(message, history):
    """
    Función principal de chat que maneja la conversación con el LLM y las herramientas.
    
    Args:
        message: Mensaje del usuario
        history: Historial de la conversación
        
    Returns:
        str: Respuesta del asistente
    """
    if not message or not message.strip():
        return "Por favor, proporciona un mensaje válido."
    
    try:
        messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
        done = False
        max_iterations = 10  # Prevenir loops infinitos
        iteration = 0
        
        while not done and iteration < max_iterations:
            iteration += 1

            # Esta es la llamada a la LLM - nota que pasamos el json de las herramientas
            response = openai.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                tools=tools
            )

            finish_reason = response.choices[0].finish_reason
            
            # Si la LLM quiere llamar a una herramienta, la llamamos!
            if finish_reason == "tool_calls":
                message = response.choices[0].message
                tool_calls = message.tool_calls
                results = handle_tool_calls(tool_calls)
                messages.append(message)
                messages.extend(results)
            else:
                done = True
        
        if iteration >= max_iterations:
            return "⚠️ Se alcanzó el límite de iteraciones. Por favor, intenta de nuevo."
        
        return response.choices[0].message.content
    except Exception as e:
        error_msg = f"Error en el chat: {str(e)}"
        print(f"❌ {error_msg}")
        return f"Lo siento, ocurrió un error: {error_msg}"

In [29]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Herramienta llamada: generate_cv_pdf_premium
⚠️ Herramienta desconocida: generate_cv_pdf_premium
Herramienta llamada: generate_cv_pdf_premium
⚠️ Herramienta desconocida: generate_cv_pdf_premium
